In [1]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

lower_purple = np.array([130,50,90])
upper_purple = np.array([170,255,255])

#Creating empty points array
points = []

# Get default camera window size
ret, frame = cap.read()
Height, Width = frame.shape[:2]
frame_count = 0

while True:
    
    # capture webcam frame
    ret, frame = cap.read()
    hsv_img = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Thresholding the HSV image to blue color
    mask = cv2.inRange(hsv_img, lower_purple, upper_purple)
    #mask = cv2.morphology(mask, cv2.MORPH_OPEN, kernel)
    
    contours,_ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Creating empty centre array to store centroid of mass
    center = int(Height/2), int(Width/2)
    
    if len(contours) > 0:
        
        c = max(contours, key=cv2.contourArea)
        (x,y), radius = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        try:
            
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        
        except:
            center = int(Height/2), int(Width/2)
        
        #We will allow only contour that have larger than 15 pixel radius
        if radius > 25:
            
            cv2.circle(frame, (int(x), int(y)), int(radius), (0,0,255), 2)
            cv2.circle(frame, center, 5, (0,255,0), -1)
            
    # log center points
    points.append(center)
    
    # Loop over the set of tracked points
    
    if radius > 25:
        for i in range(1, len(points)):
            try:
                cv2.line(frame, points[i - 1], points[i], (0,255,0), 2)
            except:
                pass
        
        frame_count = 0
    else:
        
        frame_count += 1
        
        #if we count 10 frames without object lets delete our trail
        if frame_count == 10:
            points=[]
            
            frame_count =0 
    
    # Displaying our object tracker
    frame = cv2.flip(frame, 1)
    cv2.imshow("Object Tacker", frame)
    
    if cv2.waitKey(1)==13:
        break
        
cv2.destroyAllWindows()
cap.release()

error: OpenCV(4.2.0) /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
